In [2]:
import pandas as pd
import numpy as np
from itertools import product
import os
import cv2
import tifffile as tiff
from cycifsuite.get_data import read_synapse_file
from cycifsuite.image_based_qc import *

In [4]:
imstorpath = 'Z:/sorger/data/IN_Cell_Analyzer_6000/Connor/Fixed MCF10 Common/20x full exp/20180905_Updated/'
channel_info = pd.read_csv('Z:/sorger/data/IN_Cell_Analyzer_6000/Connor/Fixed MCF10 Common/20x full exp/20180905_Updated/channel_metadata.csv')
metadata = pd.read_csv('N:/HiTS Projects and Data/Personal/Jake/mcf10a/proper_FFC_metadata.csv', index_col=0)

In [ ]:
# Update metadata with multi-nuclea cells.
finished_pwf = []
for plate in range(1,19):
    for i,j in product('CDE', ['04','05','06','07','08','09','10']):
        well = i+j
        for fld in ['fld' + str(i) for i in range(1,10)]:
            pwf = '_'.join([str(plate), well, fld])
            if pwf in finished_pwf:
                continue
            print(pwf)
            path_mask = imstorpath + 'plate' + str(plate) + '/analysis/'
            path_ffc = imstorpath + 'plate' + str(plate) + '/analysisCorrected/'
            nuc_mask = path_mask+'_'.join([well, fld, 'nucleiLM.tif'])
            cyto_mask = path_mask+'_'.join([well, fld, 'cytoLM.tif'])
            ffc = path_ffc+'_'.join([well, fld, '_FFC.tif'])
            if not os.path.exists(ffc):
                continue
            image = tiff.imread(ffc)
            img_mask = tiff.imread(nuc_mask)
            img_mask = img_mask.reshape(2048,2048)
            num_nucs, cells = find_num_cells_in_img(image, img_mask, pwf)
            if len(cells)!=metadata[metadata.pwf==pwf].shape[0]:
                num_nucs, cells = zip(*[(num_nucs[i], x) for i, x in enumerate(cells) if x in metadata.index])
            metadata.loc[cells,'num_nuclei_in_mask'] = num_nucs
            finished_pwf.append(pwf)
metadata.to_csv('N:/HiTS Projects and Data/Personal/Jake/mcf10a/proper_FFC_metadata.csv')